## Weighted and Unweighted directed graph

In [1]:
dedges = [(0,1,10),(0,2,80),(1,2,6),(1,4,20),(2,3,70),(4,5,50),(4,6,5),(5,6,10)]
size = 7

### Adjacency matrix representation

In [2]:
import numpy as np
WG = np.zeros(shape=(size,size,2))
UG = np.zeros(shape=(size,size,2))
for i,j,w in dedges:
    WG[i,j,0] = 1
    WG[i,j,1] = w
    UG[i,j,0] = 1
    UG[i,j,1] = w
    UG[j,i,0] = 1
    UG[j,i,1] = w

### Adjacency List representation

In [3]:
WL = {}
UL = {}
for i in range(size):
    WL[i] = []
    UL[i] = []
for (i,j,d) in dedges:
    WL[i].append((j,d))
    UL[i].append((j,d))
    UL[j].append((i,d))

## Djikstra Algorithm - Single source shortest path algorithm

### From one source to all other nodes 

`O(n**2) for both WG and WL`

In [4]:
def dijkstra(WMat, s):
    rows, cols, x = WMat.shape
    infinity = float('inf')
    visited, distance = {} , {}
    for v in range(rows):
        visited[v], distance[v] = False, infinity
    distance[s] = 0
    for u in range(rows):
        nextd = min(distance[v] for v in range(rows) if not visited[v])
        nextvlist = [v for v in range(rows) if not visited[v] and distance[v] == nextd]
        if nextvlist == []:
            return
        nextv = min(nextvlist)
        visited[nextv] = True
        for v in range(cols):
            if WMat[nextv, v, 0] == 1 and not visited[v]:
                distance[v] = min(distance[v], distance[nextv] + int(WMat[nextv, v, 1]))
    return distance
dijkstra(WG, 0)

{0: 0, 1: 10, 2: 16, 3: 86, 4: 30, 5: 80, 6: 35}

In [5]:
def dijkstraList(WList, s):
    infinity = float('inf')
    visited, distance = {}, {}
    for v in WList.keys():
        visited[v], distance[v] = False, infinity
    distance[s] = 0
    for u in WList.keys():
        nextd = min([distance[v] for v in WList.keys() if not visited[v]])
        nextvlist = [v for v in WList.keys() if not visited[v] and distance[v] == nextd]
        if nextvlist == []:
            return
        nextv = min(nextvlist)
        visited[nextv] = True
        for v,d in WList[nextv]:
            if not visited[v]:
                distance[v] = min(distance[v], distance[nextv] + d)
    return distance
dijkstraList(WL, 0)

{0: 0, 1: 10, 2: 16, 3: 86, 4: 30, 5: 80, 6: 35}

## Bellman Ford - Single source shortest path with negative weights

`O(n**3) for adj matrix and O(n*m) for adj List`

In [6]:
edges = [(0,1,10),(0,7,8),(1,5,2),(2,1,1),(2,3,1),(3,4,3),(4,5,-1),(5,2,-2),(6,1,-4),(6,5,-1),(7,6,1)]
size = 8
W = np.zeros(shape=(size,size,2))
for (i,j,w) in edges:
    W[i,j,0] = 1
    W[i,j,1] = w
WL = {}
for i in range(size):
    WL[i] = []
for (i,j,d) in edges:
    WL[i].append((j,d))

In [7]:
def bellmanFord(WMat, s):
    rows, cols, x = WMat.shape
    infinity = float('inf')
    distance = {}
    for v in range(rows):
        distance[v] = infinity
    distance[s] = 0
    for i in range(rows):
        for u in range(rows):
            for v in range(cols):
                if WMat[u,v,0] == 1:
                    distance[v] = min(distance[v], distance[u] + int(WMat[u,v,1]))
    return distance
bellmanFord(W, 0)

{0: 0, 1: 5, 2: 5, 3: 6, 4: 9, 5: 7, 6: 9, 7: 8}

In [8]:
def bellmanFordList(Wlist, s):
    infinity = float('inf')
    distance = {}
    for i in Wlist.keys():
        distance[i] = infinity
    distance[s] = 0
    for i in Wlist.keys():
        for u in Wlist.keys():
            for v,d in Wlist[u]:
                distance[v] = min(distance[v], distance[u] + d)
    return distance
bellmanFordList(WL, 0)

{0: 0, 1: 5, 2: 5, 3: 6, 4: 9, 5: 7, 6: 9, 7: 8}

## All pairs shortest path - Flloyd Warshal

In [9]:
def floydWarshall(WMat):
    rows, cols, x = WMat.shape
    infinity = float('inf')
    SP = np.zeros(shape=(rows,cols,cols+1))
    for i in range(rows):
        for j in range(cols):
            SP[i,j,0] = infinity
            if WMat[i,j,0] == 1:
                SP[i,j,0] = WMat[i,j,1]
    for k in range(1, cols+1):
        for i in range(rows):
            for j in range(cols):
                SP[i,j,k] = min(SP[i,j,k-1], SP[i,k-1,k-1] + SP[k-1, j, k-1])
    return SP[:,:,cols]
floydWarshall(WG)

array([[inf, 10., 16., 86., 30., 80., 35.],
       [inf, inf,  6., 76., 20., 70., 25.],
       [inf, inf, inf, 70., inf, inf, inf],
       [inf, inf, inf, inf, inf, inf, inf],
       [inf, inf, inf, inf, inf, 50.,  5.],
       [inf, inf, inf, inf, inf, inf, 10.],
       [inf, inf, inf, inf, inf, inf, inf]])

In [10]:
def floydWarshallSimple(WMat):
    rows, cols, x = WMat.shape
    infinity = float('inf')
    SP = np.zeros(shape=(rows,cols))
    for i in range(rows):
        for j in range(cols):
            if WMat[i,j,0] == 1:
                SP[i,j] = WMat[i,j,1]
            else:
                SP[i,j] = infinity
    for k in range(rows):
        for i in range(rows):
            for j in range(cols):
                SP[i,j] = min(SP[i,j], SP[i,k] + SP[k,j])
    return SP
floydWarshallSimple(WG)

array([[inf, 10., 16., 86., 30., 80., 35.],
       [inf, inf,  6., 76., 20., 70., 25.],
       [inf, inf, inf, 70., inf, inf, inf],
       [inf, inf, inf, inf, inf, inf, inf],
       [inf, inf, inf, inf, inf, 50.,  5.],
       [inf, inf, inf, inf, inf, inf, 10.],
       [inf, inf, inf, inf, inf, inf, inf]])

## Minimum Cost Spanning Tree

## Prims

In [23]:
import sys
def primList(Wlist):
    infinity = sys.maxsize + 1
    visited, distance, TreeEdges = {}, {}, []
    for v in Wlist.keys():
        visited[v],distance[v] = False, infinity
    visited[0] = True
    for v,d in Wlist[0]:
        distance[v] = d
    for i in Wlist.keys():
        mindist, nextv = infinity, None
        for u in Wlist.keys():
            for v,d in Wlist[u]:
                if visited[u] and not visited[v] and d<mindist:
                    mindist, nextv, nexte = d, v, (u,v)
        if nextv is None:
            break
        visited[nextv] = True
        TreeEdges.append(nexte)
        for v,d in Wlist[nextv]:
            if not visited[v]:
                distance[v] = min(distance[v], d)
    return TreeEdges
primList(WL)

[(0, 7), (7, 6), (6, 1), (6, 5), (5, 2), (2, 3), (3, 4)]

In [22]:
def primListBetter(Wlist):
    infinity = sys.maxsize + 1
    visited, distance, nbr = {}, {}, {}
    for v in Wlist.keys():
        visited[v], distance[v], nbr[v] = False, infinity, -1
    visited[0] = True
    for v,d in Wlist[0]:
        distance[v], nbr[v] = d, 0
    for i in range(1, len(Wlist.keys())):
        nextd = min([distance[v] for v in Wlist.keys() if not visited[v]])
        nextvlist = [v for v in Wlist.keys() if not visited[v] and distance[v] == nextd]
        if nextvlist == []:
            break
        nextv = min(nextvlist)
        visited[nextv] = True
        for v,d in Wlist[nextv]:
            if not visited[v]:
                distance[v], nbr[v] = min(distance[v], d), nextv
    return nbr
primListBetter(WL)

{0: -1, 1: 6, 2: 5, 3: 2, 4: 3, 5: 1, 6: 7, 7: 0}

## Krushkal Algorithm (O^2)

In [26]:
def kruskal(Wlist):
    edges, component, TE = [], {}, []
    for u in Wlist.keys():
        edges.extend([(d,u,v) for v,d in Wlist[u]])
        component[u] = u
    edges.sort()
    for d,u,v in edges:
        if component[u] != component[v]:
            TE.append((u,v))
            c = component[u]
            for w in Wlist.keys():
                if component[w] == c:
                    component[w] = component[v]
    return TE
kruskal(WL)

[(6, 1), (5, 2), (4, 5), (6, 5), (2, 3), (7, 6), (0, 7)]